In [28]:
from wiggle import CarbonaraDataTools as cdt
from wiggle import writhe_evo as wgl
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm

In [18]:
# Creating functions that read a pdb and generate the StS and StA writhe metrics

def StS_from_pdb(pdb_file, stride=1):

    coords = cdt.extract_coords_from_PDB(pdb_file)
    return wgl.find_Sigma_array(coords[::stride])

def StA_from_pdb(pdb_file, stride=1):

    coords = cdt.extract_coords_from_PDB(pdb_file)
    return wgl.calculate_segment_to_all_from_points(coords[::stride])

In [67]:
pdb_files = np.sort(glob('../data/random_lysosome_conformations/AA_pdbs/*'))

file_names = []
StA_arr = np.zeros([len(pdb_files), StA_from_pdb(pdb_files[0], stride=4).shape[0]])


for i, file in tqdm( enumerate(pdb_files) ):

    file_names.append( file.split('/')[-1].split('.')[0] )
    StA_arr[i,:] = StA_from_pdb(file, stride=4)

9858it [03:11, 51.59it/s]


In [134]:
df = pd.DataFrame(StA_arr)

col_names = []
for c in df.columns:
    col_names.append('segment_'+str(c))

df.columns = col_names

file_names_appen = []
for f in file_names:
    file_names_appen.append(f+'.pdb')

df['file_name'] = file_names_appen

energy_df = pd.read_csv('../data/random_lysosome_conformations/conformation_energy.csv')

df_StA_energy = df.merge(energy_df, on='file_name', how='inner')

In [136]:
# df_StA_energy[ df_StA_energy['Energy'].values>1e5 ]

In [137]:
df_StA_energy = df_StA_energy[ df_StA_energy['Energy'].values<1e5 ]

In [138]:
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import shap

In [139]:
X = df_StA_energy.drop(columns=['file_name', 'Energy'])
y = df_StA_energy['Energy']

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [141]:
params = { 'iterations': 1000,
            'learning_rate': 0.05,
            # 'cat_features':self.cat_feats,
            'depth': 7,
            'bagging_temperature':1,
            'l2_leaf_reg':1,
            'verbose':200,
            #'best_model_min_trees':25,
            'od_type':"IncToDec", # overfit detector
            'od_wait':500, # most recent best iteration to wait before stopping
            'random_seed': 101,
            }
    
    
cat_model = CatBoostRegressor(**params, allow_writing_files=False)
cat_model.fit(X_train, y_train,   
                eval_set=(X_test, y_test), 
                use_best_model=True,
                plot=False,
                verbose_eval=False,
                );

In [142]:
cat_model.best_score_['validation']

{'RMSE': 4728.959100885514}

In [143]:
y.describe()

count     9830.000000
mean     50156.502299
std       4606.069681
min      39365.769911
25%      47412.950668
50%      49438.940480
75%      51880.141398
max      94165.372230
Name: Energy, dtype: float64